In [1]:
import os
import numpy as np
import tensorflow as tf
from  tensorflow import keras
from matplotlib import pyplot as plt
from tensorflow.keras import Sequential, layers
from    PIL import Image

In [2]:
# 设置随机种子以及提示信息
tf.random.set_seed(22)
np.random.seed(22)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
assert tf.__version__.startswith('2.')

In [17]:
def save_images(imgs,name):
    new_img = Image.new('L',(280,280))

    index = 0
    for i in range(0,280,28):
        for j in range(0,280,28):
            im = imgs[index]
            im = Image.fromarray(im,mode ='L')
            new_img.paste(im,(i,j))
            index +=1

    new_img.save(name)

In [5]:
# 超参数设置
h_dim = 20
batch_size = 512
lr = 1e-3

In [7]:
(x_train,y_train),(x_test,y_test) = keras.datasets.fashion_mnist.load_data()

# image preprocess
x_train,x_test = x_train.astype(np.float32)/255.,x_test.astype(np.float32)/255.

train_db = tf.data.Dataset.from_tensor_slices(x_train)
train_db = train_db.shuffle(batch_size*5).batch(batch_size)
test_db = tf.data.Dataset.from_tensor_slices(x_test)
test_db = test_db.batch(batch_size)

print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [9]:
class AE(keras.Model):
    def __init__(self):
        super(AE,self).__init__()
        # encoder
        self.encoder = Sequential([
            layers.Dense(256,activation=tf.nn.relu),
            layers.Dense(128,activation=tf.nn.relu),
            layers.Dense(h_dim)
        ])

        # decoder
        self.decoder = Sequential([
            layers.Dense(128,activation=tf.nn.relu),
            layers.Dense(256,activation=tf.nn.relu),
            layers.Dense(784)
        ])

    def call(self,inputs,training=None):
        h = self.encoder(inputs)
        x_hat = self.decoder(h)

        return x_hat


In [13]:
model = AE()
model.build(input_shape=(None,784))
model.summary()

Model: "ae_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_2 (Sequential)    (None, 20)                236436    
_________________________________________________________________
sequential_3 (Sequential)    (None, 784)               237200    
Total params: 473,636
Trainable params: 473,636
Non-trainable params: 0
_________________________________________________________________


In [ ]:
optimizer = tf.optimizers.Adam(lr=lr)

for epoch in range(100):
    for step,x in enumerate(train_db):

        x = tf.reshape(x,[-1,784])
        with tf.GradientTape() as tape:
            x_rec_logits = model(x)

            rec_loss = tf.losses.binary_crossentropy(x,x_rec_logits,from_logits = True)
            rec_loss = tf.reduce_mean(rec_loss)

        grads = tape.gradient(rec_loss,model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if step %100 == 0:
            print(epoch,step,float(rec_loss))

    # evaluation
    x = next(iter(test_db))
    logits = model(tf.reshape(x, [-1, 784]))
    x_hat = tf.sigmoid(logits)
    # [b, 784] => [b, 28, 28]
    x_hat = tf.reshape(x_hat, [-1, 28, 28])

    # [b, 28, 28] => [2b, 28, 28]
    x_concat = tf.concat([x, x_hat], axis=0)
    x_concat = x_hat
    x_concat = x_concat.numpy() * 255.
    x_concat = x_concat.astype(np.uint8)
    save_images(x_concat, 'ae_images/rec_epoch_%d.png'%epoch)

        